In [1]:

import scipy
import numpy as np
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import scipy.constants as const
from scipy.spatial.transform import Rotation as R

## Constants

In [11]:
# Valor de bohr en metros
bohr_factor = const.value('Bohr radius')
# Factor de conversión de angstrom a bohr
angstrom_to_bohr = const.angstrom / bohr_factor
hartree_to_kcal = 627.509
kJ_to_kcal = 1 / 4.184

# Data

In [5]:
# Data from sapt2

distances_r = [3.2, 3.3, 3.4, 3.6, 3.7, 3.8, 3.9, 4.0, 4.2, 4.4, 4.5, 4.6, 4.7,
               4.8, 4.9, 5.2, 5.4, 5.5, 5.7, 6.1, 6.3, 6.5, 6.8]

energies = [1.24707, 0.92268, 0.6865,  0.39661, 0.31076, 0.24977, 0.20653,
            0.17591, 0.13882, 0.11991, 0.11411, 0.10977, 0.10641, 0.10371,
            0.10146, 0.09609, 0.09307, 0.09164, 0.08891, 0.08384, 0.08148,
            0.07922, 0.07603]


ind_disp = [-160.83328237, -117.40916117, -88.02454154, -51.32540357,
            -39.99912747, -31.65553907, -25.4411985, -20.74822247,
            -14.342125240000001, -10.31628075, -8.842262009999999,
            -7.62124806, -6.60364195, -5.7528848, -5.03997461, -3.50615447,
            -2.82605019, -2.55482163, -2.1133328799999997, -1.50350457,
            -1.2874137600000002, -1.11120684, -0.90249682]

# Data from isa
charge_by_atom = [-0.547209,  -0.547209,  -0.547209,  -0.542063, -0.542063,
                  -0.552955,  2.272009]


base_molecule = np.array([[-0.287885, 1.142383, 1.086087],
                          [-0.287885, -1.142383, 1.086087],
                          [-1.520467, 0.0, -0.456775],
                          [0.325382, -1.131606, -1.059753],
                          [0.325382, 1.131606, -1.059753],
                          [1.548288, 0.0, 0.428623],
                          [0.0, 0.0, 0.0]])

In [6]:
energies_kcal = (np.array(energies)*627.509)
x_values = np.array(distances_r)
y_values = np.array(ind_disp)
atom_type = [1, 1, 1, 1, 1, 1, 0]

In [7]:
atom_type = np.array(atom_type)
atom_type2 = np.array(atom_type)
atom_type = atom_type[:, np.newaxis]
atom_type2 = atom_type2[np.newaxis, :]
type_matrix = atom_type + atom_type2
type_matrix_flat = type_matrix.flatten()

# optimization

In [2]:
from scipy import optimize

#### Function for optimization or calculate differences

In [17]:
def function_test(solution):
    p_v = solution
    vec_1a = [p_v[0],p_v[0],p_v[0],p_v[0],p_v[0],p_v[0],p_v[0],p_v[3],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[9]]
    vec_1b = [p_v[1],p_v[1],p_v[1],p_v[1],p_v[1],p_v[1],p_v[1],p_v[4],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[10]]
    vec_1c = [p_v[2],p_v[2],p_v[2],p_v[2],p_v[2],p_v[2],p_v[2],p_v[5],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[11]]
    vec_1a = np.array(vec_1a)
    vec_1b = np.array(vec_1b)
    vec_1c = np.array(vec_1c)
    vec_2a = [p_v[12],p_v[12],p_v[12],p_v[12],p_v[12],p_v[12],p_v[15]]
    vec_2b = [p_v[13],p_v[13],p_v[13],p_v[13],p_v[13],p_v[13],p_v[16]]
    vec_2c = [p_v[14],p_v[14],p_v[14],p_v[14],p_v[14],p_v[14],p_v[17]]
    vec_2a = np.array(vec_2a)
    vec_2b = np.array(vec_2b)
    vec_2c = np.array(vec_2c)

    v_12a = np.array((np.outer(vec_1a, vec_2a).flatten())**(0.5))
    v_12b = np.array((np.outer(vec_1b, vec_2b).flatten())**(0.5))
    v_12c = np.array(((0.5)*(vec_1c[:, np.newaxis] + vec_2c)).flatten())

    vec_d = total_distances ## desde fuera
    vec_a, vec_b, vec_c = v_12a, v_12b, v_12c
    vec_q1q2 = vec_q1q2_pf6_quin ### desde fuera
    
   #### Transformar las energias objetivo 

    combination_matrix = []
    for vector in vec_d:
        combination_matrix.append(np.vstack((vector, vec_a, vec_b, vec_c, vec_q1q2)))

    combination_matrix = np.array(combination_matrix)
    energies = []
    for i in range(combination_matrix.shape[0]):
        sum = 0
        for j in range(combination_matrix.shape[2]):
            exp_term = (combination_matrix[i, 2, j])*(np.exp((-combination_matrix[i, 3, j])*(combination_matrix[i, 0, j])))
            dis_ind_term = combination_matrix[i, 1, j]/((combination_matrix[i, 0, j])**6)
            elst_term = (combination_matrix[i, 4, j])/(combination_matrix[i, 0, j]*angstrom_to_bohr)
            sum = sum + (exp_term*kJ_to_kcal) - (dis_ind_term*kJ_to_kcal) + (elst_term*hartree_to_kcal)
        energies.append(sum)
        
    diferencias = energies - total_energies
    diferencias = np.array(diferencias)
    diferencias = diferencias.sum

    return 

Other functions

In [15]:
coordinates_quin_to_pf6_1 = np.load (r"C:\Users\migue\Downloads\Pf6_quin_configurations_genetic.npy")
coordinates_quin_to_pf6_2 = np.load (r"C:\Users\migue\Downloads\quin_configurations_genetic.npy")

coordinates_quin_to_pf6_total = np.concatenate((coordinates_quin_to_pf6_1, coordinates_quin_to_pf6_2), axis = 0)


# first coordinates pf6 - 80 : "C:\Users\migue\Downloads\PF6_centered_for_geneticAlg.xyz"
# second coordinates pf6 - 18: "C:\Users\migue\Downloads\pf6_centered_180_1_genetic.xyz"
# thrid coordinates pf6 - 18: "C:\Users\migue\Downloads\pf6_centered_180_2_genetic.xyz"
# forth coordinates pf6 - 18: "C:\Users\migue\Downloads\pf6_centered_180_3_genetic.xyz"


def xyz_a_matriz_numpy(ruta_archivo_xyz):
    # Leer el contenido del archivo
    with open(ruta_archivo_xyz, 'r') as archivo:
        lineas = archivo.readlines()

    # Omitir la primera línea (contiene el número total de átomos) y otras no necesarias
    # Asumimos que las coordenadas comienzan desde la tercera línea
    lineas_coordenadas = lineas[2:]
    
    # Preparar una lista para almacenar las coordenadas
    coordenadas = []
    
    for linea in lineas_coordenadas:
        if linea.strip():  # Verificar que la línea no esté vacía
            partes = linea.split()
            if len(partes) == 4:  # Asegurar que la línea tenga el formato esperado (Elemento x y z)
                # Convertir las partes numéricas (x, y, z) a flotantes y añadir a la lista
                coordenadas.append([float(partes[1]), float(partes[2]), float(partes[3])])

    # Convertir la lista de coordenadas a una matriz NumPy
    matriz_coordenadas = np.array(coordenadas)

    return matriz_coordenadas


coord_pf6_80 = xyz_a_matriz_numpy(r"C:\Users\migue\Downloads\PF6_centered_for_geneticAlg.xyz")
coord_pf6_18_1 = xyz_a_matriz_numpy(r"C:\Users\migue\Downloads\pf6_centered_180_1_genetic.xyz")
coord_pf6_18_2 = xyz_a_matriz_numpy(r"C:\Users\migue\Downloads\pf6_centered_180_2_genetic.xyz")
coord_pf6_18_3 = xyz_a_matriz_numpy(r"C:\Users\migue\Downloads\pf6_centered_180_3_genetic.xyz")


def space_distances_generation(base_molecule, configurations):
    space_distance_vec = []
    for w in range(len(configurations)):  # w is the distances vector
        space_distance = cdist(base_molecule, configurations[w], 'euclidean')
        space_distance = np.array(space_distance.flatten())
        space_distance_vec.append(space_distance)
    return np.array(space_distance_vec)

energies_80 = np.load(r"C:\Users\migue\Downloads\pf6_quin_energies_80.npz")
energies_180_1 = np.load(r"C:\Users\migue\Downloads\quin_pf6_180_1_ene.npz")
energies_180_2 = np.load(r"C:\Users\migue\Downloads\quin_fp6_180_2_ene.npz")
energies_180_3 = np.load(r"C:\Users\migue\Downloads\quin_pf6_180_3_ene.npz")

energies_pf6_80 = energies_80['esapt']
energies_pf6_180_1 = energies_180_1['esapt']*hartree_to_kcal
energies_pf6_180_2 = energies_180_2['esapt']*hartree_to_kcal
energies_pf6_180_3 = energies_180_3['esapt']*hartree_to_kcal

total_energies = np.array(np.concatenate((energies_pf6_80,energies_pf6_180_1,energies_pf6_180_2, energies_pf6_180_3), axis = 0))

distances_80 = space_distances_generation(coord_pf6_80, coordinates_quin_to_pf6_1)
distances_180_1 = space_distances_generation(coord_pf6_18_1, coordinates_quin_to_pf6_2)
distances_180_2 = space_distances_generation(coord_pf6_18_2, coordinates_quin_to_pf6_2)
distances_180_3 = space_distances_generation(coord_pf6_18_3, coordinates_quin_to_pf6_2)

total_distances = np.concatenate((distances_80, distances_180_1, distances_180_2, distances_180_3), axis = 0)


####concatenar distancias 

charges_chn = [0.038097, -0.167411, -0.167412, 0.030591, -0.013700, -0.013700, -0.234656,
           0.272792, 0.094346, 0.097032,  0.097032, 0.098344, 0.098344, 0.079640, 0.079641,
           0.092649, 0.092649, 0.091285, 0.091285, 0.103953, 0.103953, 0.035246]


charges_pf6 = [-0.547209,  -0.547209,  -0.547209,  -0.542063, -0.542063,
               -0.552955,  2.272009]


# a b c for molecules C(4), H(4), H(1), N(3), F, P
p_v = [978.36, 131571, 3.60, 0.00, 764.9, 3.56, 278.37, 12680, 3.56,
                 2376.55, 191935, 3.48, 283248.0316091423, 273570.9941372, 4.0775266,
                 283248.0316091423, 273570.9941372, 4.0775266]

def vector_q1q2(vec_1, vec_2):
    v_q1q2 = np.outer(vec_1, vec_2).flatten()
    return np.array(v_q1q2)


vec_q1q2_pf6_quin = vector_q1q2(charges_pf6, charges_chn)

Energies results

In [12]:
energies_80 = np.load(r"C:\Users\migue\Downloads\pf6_quin_energies_80.npz")
energies_180_1 = np.load(r"C:\Users\migue\Downloads\quin_pf6_180_1_ene.npz")
energies_180_2 = np.load(r"C:\Users\migue\Downloads\quin_fp6_180_2_ene.npz")
energies_180_3 = np.load(r"C:\Users\migue\Downloads\quin_pf6_180_3_ene.npz")

energies_pf6_80 = energies_80['esapt']
energies_pf6_180_1 = energies_180_1['esapt']*hartree_to_kcal
energies_pf6_180_2 = energies_180_2['esapt']*hartree_to_kcal
energies_pf6_180_3 = energies_180_3['esapt']*hartree_to_kcal

total_energies = np.array(np.concatenate((energies_pf6_80,energies_pf6_180_1,energies_pf6_180_2, energies_pf6_180_3), axis = 0))


x0 = [978.36, 131571, 3.60, 0.00, 764.9, 3.56, 278.37, 12680, 3.56,
                 2376.55, 191935, 3.48, 824.667546247, 803364.870629, 6.14391455469,
                 2194.9694610908, 126176948.8657, 3.250978952232]

In [18]:
vector_solution = scipy.optimize.minimize(function_test, x0 )

TypeError: '<' not supported between instances of 'NoneType' and 'float'